In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [3]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## Rule-Based Bots ##

In [90]:
config = GameConfig(splits=[StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit], sandbox=True, num_players=2, must_include=[])
players = load_players(['BM', 'BM'], None, map_location='cpu', train=False)
agent = players[0]
env = FullEnvironment(config, players, logger=logger)

In [91]:
n = 10000

In [92]:
sim_stats = simulate(env, n, None, turn_log=True, action_log=True)

100%|██████████| 10000/10000 [03:08<00:00, 53.01it/s]
===SUMMARY===
{0: 5716, 1: 5574, 'ProvinceWins': 10000, 'Ties': 1290}


In [95]:
data_path = os.path.join(project_root, 'data', 'bm-bm-10k-stats')

In [96]:
save(data_path, sim_stats)

## Tree Simulations ##

In [8]:
from mcts import GameTree

In [6]:
# tree_name = 'full-paper-tree-uniform'
# tree_name = 'r-r-mcts-bm-full-100k-srat-uniform-paper'
# tree_name = 'r-dw-mcts-bm-full-160k-srat-uniform-paper'
tree_name = 'full-paper-tree-160k-trimmed-lowc'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [38]:
# tree_name = 'default-sandbox-uct-paper'
# tree_name = 'default-sandbox-uct-paper'
# tree_name = 'r-r-mcts-bm-hot-start-sandbox-100k-srat-uniform'
rollout_name = 'default-mlog-sandbox-paper'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [9]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [36]:
tree.selection = 'robust'
tree.selection

'robust'

In [67]:
tree.skip_level = False

In [35]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
# splits = [StartingSplit.Starting34Split, StartingSplit.Starting25Split]
splits = [StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit]
# config = GameConfig(splits=splits, sandbox=True, num_players=2, must_include=[])
config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = load_players(['UCT', 'BM'], [rollout_path], tree=tree, rollout_type='mlog', train=False, use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)[0]
players = [agent, agent]

env = DefaultEnvironment(config, players, logger=logger)

In [30]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [36]:
sim_data = simulate(env, 1000, tree, turn_log=False, action_log=False, card_log=False)
# sim_data = simulate(env, 1, tree, turn_log=False, action_log=False, card_log=False)

100%|██████████| 1000/1000 [00:26<00:00, 38.31it/s]


===SUMMARY===
{0: 578, 1: 436, 'ProvinceWins': 974, 'Ties': 14}


In [11]:
data_dir = os.path.join(project_root, 'data')
data_name ='ucthot25-bm25-1k-data'
data_path = os.path.join(data_dir, data_name)

In [12]:
save(data_path, sim_data)